In [1]:
def znajdz_najwieksza(lista):
    if not lista:
        raise ValueError("Lista jest pusta")
    najwieksza = lista[0]
    for liczba in lista[1:]:
        if liczba > najwieksza:
            najwieksza = liczba
    return najwieksza

# Przykład użycia:
moje_liczby = [10, 5, 7, 22, 3]
print(znajdz_najwieksza(moje_liczby))  # Wynik: 22

22


In [3]:
import random
import networkx as nx

class NarrativeMap:
    """
    Klasa do budowy interaktywnych map narracyjnych w postaci drzewa decyzyjnego dla gier fabularnych.

    :param themes: Lista motywów przewodnich (str).
    :param locations: Lista dostępnych lokacji (str).
    :param characters: Lista postaci (str).
    """

    def __init__(self, themes, locations, characters):
        """
        Inicjalizuje mapę narracyjną na podstawie motywów, lokacji i postaci.
        """
        self.themes = themes
        self.locations = locations
        self.characters = characters
        self.graph = nx.DiGraph()
        self._build_decision_tree()

    def _build_decision_tree(self):
        """
        Buduje drzewo decyzyjne z losowo generowanymi zdarzeniami, gałęziami oraz parametrami ryzyka/nagrody.
        """
        # Startowy węzeł
        root_id = 0
        self.graph.add_node(root_id, description=self._generate_event_description(root_id),
                            risk=0, reward=0)
        node_queue = [(root_id, 0)]

        max_depth = 3
        node_id = 1

        for depth in range(max_depth):
            new_queue = []
            for parent_id, _ in node_queue:
                # Dwa możliwe wybory na każdym poziomie
                for choice_label in ["zaufaj", "zdrada"] if depth % 2 == 0 else ["walcz", "ucieczka"]:
                    risk = random.randint(1, 10)
                    reward = random.randint(1, 10)
                    self.graph.add_node(node_id, description=self._generate_event_description(node_id),
                                        risk=risk, reward=reward)
                    self.graph.add_edge(parent_id, node_id, choice=choice_label)
                    new_queue.append((node_id, depth + 1))
                    node_id += 1
            node_queue = new_queue

    def _generate_event_description(self, node_id):
        """
        Generuje opis zdarzenia w oparciu o themes, locations i characters.
        (W docelowej wersji wywołuje GPT lub inny generator.)
        """
        # TODO: call GPT here
        theme = random.choice(self.themes)
        location = random.choice(self.locations)
        character = random.choice(self.characters)
        sentences = [
            f"W {location} pojawia się {character}.",
            f"Tajemnicza sytuacja związana z motywem: {theme}.",
            f"Wybór zmieni losy bohaterów."
        ]
        return " ".join(random.sample(sentences, 2 + (node_id % 2)))  # 2–3 zdania

    def get_path_summary(self, path):
        """
        Zwraca podsumowanie ścieżki wraz z sumą ryzyka i nagrody.

        :param path: Lista ID węzłów (int) określająca przebieg ścieżki.
        :return: Słownik z opisem, sumą ryzyka i nagrody oraz opisami wyborów.
        """
        summary = []
        total_risk = 0
        total_reward = 0
        for i, node in enumerate(path):
            node_data = self.graph.nodes[node]
            desc = node_data["description"]
            risk = node_data["risk"]
            reward = node_data["reward"]
            total_risk += risk
            total_reward += reward
            if i > 0:
                choice = self.graph.edges[path[i-1], node]["choice"]
                summary.append(f"Wybrano: {choice}")
            summary.append(f"[Węzeł {node}] {desc} (Ryzyko: {risk}, Nagroda: {reward})")
        return {
            "opis": "\n".join(summary),
            "suma_ryzyka": total_risk,
            "suma_nagrody": total_reward
        }

    def example_usage(self):
        """
        Przykładowe użycie: generuje i prezentuje przykładową ścieżkę oraz jej podsumowanie.
        """
        root = 0
        # Przechodzimy w dół wybierając zawsze pierwszy dostępny węzeł
        path = [root]
        curr = root
        while True:
            children = list(self.graph.successors(curr))
            if not children:
                break
            curr = children[0]
            path.append(curr)
        summary = self.get_path_summary(path)
        print("Opis przebiegu ścieżki:")
        print(summary["opis"])
        print(f"SUMA RYZYKA: {summary['suma_ryzyka']} | SUMA NAGRODY: {summary['suma_nagrody']}")


if __name__ == "__main__":
    # Przykład użycia:
    themes = ["odkupienie", "tajemnica"]
    locations = ["opuszczony zamek", "mglista dolina"]
    characters = ["Wędrowiec", "Cień"]

    mapa = NarrativeMap(themes, locations, characters)
    mapa.example_usage()

Opis przebiegu ścieżki:
[Węzeł 0] W mglista dolina pojawia się Wędrowiec. Wybór zmieni losy bohaterów. (Ryzyko: 0, Nagroda: 0)
Wybrano: zaufaj
[Węzeł 1] W opuszczony zamek pojawia się Wędrowiec. Wybór zmieni losy bohaterów. Tajemnicza sytuacja związana z motywem: odkupienie. (Ryzyko: 1, Nagroda: 6)
Wybrano: walcz
[Węzeł 3] Wybór zmieni losy bohaterów. Tajemnicza sytuacja związana z motywem: odkupienie. W opuszczony zamek pojawia się Cień. (Ryzyko: 7, Nagroda: 3)
Wybrano: zaufaj
[Węzeł 7] W opuszczony zamek pojawia się Cień. Tajemnicza sytuacja związana z motywem: tajemnica. Wybór zmieni losy bohaterów. (Ryzyko: 9, Nagroda: 1)
SUMA RYZYKA: 17 | SUMA NAGRODY: 10


In [2]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.datasets import fetch_california_housing
data = fetch_california_housing()
X,y = data.data, data.target

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [6]:
from sklearn.metrics import mean_squared_error

with mlflow.start_run():
    model = RandomForestRegressor(n_estimators=100)
    model.fit(X_train,y_train)
    mlflow.log_param("n_estimators",100)
    mlflow.log_metric("mse",mean_squared_error(y_test,model.predict(X_test)))

    mlflow.sklearn.log_model(model,"model")

2025/07/22 08:35:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/22 08:35:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
